# Basics of Agents
We shall look at the basics of the Chat Completions API and prompting which are the underpinnings of 
Agentic operations such as tool calls.

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from openai import OpenAI
client = OpenAI()

# https://platform.openai.com/docs/api-reference/chat/create

TEST_PROMPT = """
You are an helpful AI Assistant, with the ability to fetch restaurants and answer questions based on provided restaurant
reviews. Use the tool calling to fetch reviews for a given restaurant requested by the user.
When the reviews are given as a list, copy them to the output for the next AI assistant to work on. Do not summarize.

Tool Call: fetch_review(restaurant_name) where restaurant_name is the name of the restaurant. Do not output anything else.
"""


completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": TEST_PROMPT},
        {
            "role": "user",
            "name": "Joe",
            "content": "Fetch reviews for Starbucks."
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='fetch_review(Starbucks)', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [4]:
print(completion.choices[0].message.content)

fetch_review(Starbucks)


In [5]:
import agentops

from autogen import ConversableAgent

In [6]:
agentops.init(os.environ.get("AGENTOPS_API_KEY"), default_tags=["simple-autogen-example"])

# Microsoft Autogen: ConversableAgent: https://microsoft.github.io/autogen/0.2/docs/tutorial/introduction/
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

🖇 AgentOps:  WARNING: agentops is out of date. Please update with the command: 'pip install --upgrade agentops'
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=808a54dc-3353-4c15-b37d-81e50795779d


In [7]:
chat_result = joe.initiate_chat(cathy, message="Tell me a joke", max_turns=2)

# Close your AgentOps session to indicate that it completed.
agentops.end_session("Success")

joe (to cathy):

Tell me a joke

--------------------------------------------------------------------------------
cathy (to joe):

Sure, here's one: 

Why don't we ever tell secrets on a farm?

Because the potatoes have eyes, the corn has ears, and the beans stalk!

--------------------------------------------------------------------------------
joe (to cathy):

Good one, Cathy! Here's another one for you:

Why don’t scientists trust atoms?

Because they make up everything!

--------------------------------------------------------------------------------
cathy (to joe):

Oh, Joe, that's a good one! A little science humor. Here's another chemistry joke for you:

Why can't you trust an atom that's been split?

Because it's gone fission!

--------------------------------------------------------------------------------


🖇 AgentOps: Session Stats - Duration: 5.4s | Cost: $0.006480 | LLMs: 3 | Tools: 0 | Actions: 16 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=808a54dc-3353-4c15-b37d-81e50795779d


## Lab 1: Basic functions
Functions below load restaurant reviews and fetch reviews for a given restaurant.

In [8]:
from typing import Annotated, Literal
from typing import Dict
from typing import List
restaurant_datafile="restaurant-data.txt"


def load_restaurant_reviews() -> Dict[str, List[str]]:
    restaurant_map :Dict[str, List[str]] = {}
    with open(restaurant_datafile) as file:
        for line in file:
            # print(line)
            splits = line.split(".")
             # Use setdefault to ensure the key has a list, then append the value
            restaurant_map.setdefault(splits[0], []).append('.'.join(splits[1:]))
    # Iterate through the dictionary
    # for key, value in restaurant_map.items():
    #    print(f"Key: {key}, Value: {value}")
    return restaurant_map
    
def fetch_restaurant_reviews(restaurant_name: Annotated[str, "restaurant name"]) -> Dict[str, List[str]]:
    reviews = load_restaurant_reviews()
    return {restaurant_name: reviews[restaurant_name]}

In [9]:
fetch_restaurant_reviews("Starbucks")

{'Starbucks': [' Starbucks consistently serves good coffee and tasty pastries. The customer service is enjoyable, with baristas who are usually friendly and efficient.\n',
  ' The coffee was good and consistently prepared. The baristas provided amazing customer service, being both friendly and efficient.\n',
  ' Starbucks consistently delivers good coffee and pastries. The customer service was incredible, with friendly baristas who remembered my usual order.\n',
  " Starbucks consistently serves good coffee and tasty pastries. The customer service is awesome, with friendly baristas who remember regular customers' orders.\n",
  ' Both the food and service at Starbucks were great. The coffee was satisfying, and the barista was incredibly friendly and efficient.\n',
  ' Starbucks consistently serves good coffee and tasty snacks. The customer service is incredible, with friendly baristas who often remember regular customers.\n',
  ' Starbucks consistently delivers good coffee and pastries.

## Lab 1: Part 1 Two agent design to fetch reviews
Two Autogen ConversibleAgents to fetch reviews. We need a two agent design since one is a simple user agent used to
represent the user's queries. The second one is an LLM agent, using "gpt-4" as the LLM. Example shows "tool use" by registering
the tool description with the LLM agent, so that it can output a tool use call. The user agent receives that call and is able to execute the tool.

In [10]:
ENTRYPOINT_AGENT_PROMPT = """
You are an helpful AI Assistant, with the ability to fetch restaurants and answer questions based on provided restaurant
reviews. Use the tool calling to fetch reviews for a given restaurant requested by the user.
When the reviews are given as a list, copy them to the output for the next AI assistant to work on. Do not summarize.
"""

agentops.start_session()

restaurant_review_agent = ConversableAgent("Restaurant Review agent",
    system_message=ENTRYPOINT_AGENT_PROMPT,
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)


# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)



🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=556fca72-dcf9-4575-8fb0-d9147aaa2b75


In [11]:
# Tool use: https://microsoft.github.io/autogen/0.2/docs/tutorial/tool-use/
# Register the tool signature with the assistant agent.
restaurant_review_agent.register_for_llm(name="fetch_restaurant_reviews",
                                         description="Fetch reviews for a given restaurant name.")(fetch_restaurant_reviews)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="fetch_restaurant_reviews")(fetch_restaurant_reviews)

<function __main__.fetch_restaurant_reviews(restaurant_name: typing.Annotated[str, 'restaurant name']) -> Dict[str, List[str]]>

In [12]:
# Termination: use max_turns = 2
chat_result = user_proxy.initiate_chat(restaurant_review_agent, message="Fetch reviews for Starbucks.", max_turns=2)
agentops.end_session("Success")

User (to Restaurant Review agent):

Fetch reviews for Starbucks.

--------------------------------------------------------------------------------
Restaurant Review agent (to User):

***** Suggested tool call (call_SlJa6rpOV4dSosgR62O0NW69): fetch_restaurant_reviews *****
Arguments: 
{
  "restaurant_name": "Starbucks"
}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION fetch_restaurant_reviews...
User (to Restaurant Review agent):

User (to Restaurant Review agent):

***** Response from calling tool (call_SlJa6rpOV4dSosgR62O0NW69) *****
{"Starbucks": [" Starbucks consistently serves good coffee and tasty pastries. The customer service is enjoyable, with baristas who are usually friendly and efficient.\n", " The coffee was good and consistently prepared. The baristas provided amazing customer service, being both friendly and efficient.\n"

🖇 AgentOps: Session Stats - Duration: 13.7s | Cost: $0.060330 | LLMs: 3 | Tools: 1 | Actions: 15 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=556fca72-dcf9-4575-8fb0-d9147aaa2b75


## Part 2: Compute a score for each review
We use the "Sequence" chat pattern from Autogen to send the fetched reviews to a scoring agent.
Example below shows how to send these to the food scoring agent. The prompt uses the food scoring logic to compute
a score.

In [13]:
# Sequence patterns: https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns
FOODSCORE_AGENT_PROMPT = """
You are an helpful AI Assistant, with the ability to analyze and rate (from 1 to 5) the food service at a restaurant given the reviews.
Use the following table:
Score 1/5 has one of these adjectives: awful, horrible, or disgusting.
Score 2/5 has one of these adjectives: bad, unpleasant, or offensive.
Score 3/5 has one of these adjectives: average, uninspiring, or forgettable.
Score 4/5 has one of these adjectives: good, enjoyable, or satisfying.
Score 5/5 has one of these adjectives: awesome, incredible, or amazing.
Given a review, use the above table to output a rating as an integer. Only output the integer and nothing else.
"""

food_score_agent = ConversableAgent("Food Score agent",
    system_message=FOODSCORE_AGENT_PROMPT,
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

🖇 AgentOps: Could not create agent - no sessions detected. Create a session by calling agentops.start_session()


In [14]:
chat_result = user_proxy.initiate_chats(
[
        {
            "recipient": restaurant_review_agent,
            "message": "Whats the overall score for Starbucks ?",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": food_score_agent,
            "message": "Given these reviews, generate a score for the food.",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
])


********************************************************************************
Starting a new chat....

********************************************************************************


🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()


User (to Restaurant Review agent):

Whats the overall score for Starbucks ?

--------------------------------------------------------------------------------


🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()


Restaurant Review agent (to User):

***** Suggested tool call (call_MS1CYsivOetITdyDB5L5Ce48): fetch_restaurant_reviews *****
Arguments: 
{
  "restaurant_name": "Starbucks"
}
*****************************************************************************************

--------------------------------------------------------------------------------


...

Restaurant Review agent (to User):

Here are the reviews for Starbucks:

1. " Starbucks consistently serves good coffee and tasty pastries. The customer service is enjoyable, with baristas who are usually friendly and efficient."
2. " The coffee was good and consistently prepared. The baristas provided amazing customer service, being both friendly and efficient."
3. " Starbucks consistently delivers good coffee and pastries. The customer service was incredible, with friendly baristas who remembered my usual order."
4. " Starbucks consistently serves good coffee and tasty pastries. The customer service is awesome, with friendly baristas who remember regular customers' orders."
5. " Both the food and service at Starbucks were great. The coffee was satisfying, and the barista was incredibly friendly and efficient."
6. " Starbucks consistently serves good coffee and tasty snacks. The customer service is incredible, with friendly baristas who often remember regular customers."
7. " Starbuck

🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()


User (to Food Score agent):

Given these reviews, generate a score for the food.
Context: 
Here are the reviews for Starbucks:

1. " Starbucks consistently serves good coffee and tasty pastries. The customer service is enjoyable, with baristas who are usually friendly and efficient."
2. " The coffee was good and consistently prepared. The baristas provided amazing customer service, being both friendly and efficient."
3. " Starbucks consistently delivers good coffee and pastries. The customer service was incredible, with friendly baristas who remembered my usual order."
4. " Starbucks consistently serves good coffee and tasty pastries. The customer service is awesome, with friendly baristas who remember regular customers' orders."
5. " Both the food and service at Starbucks were great. The coffee was satisfying, and the barista was incredibly friendly and efficient."
6. " Starbucks consistently serves good coffee and tasty snacks. The customer service is incredible, with friendly barist

🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()
🖇 AgentOps: Could not record event - no sessions detected. Create a session by calling agentops.start_session()


Food Score agent (to User):

5

--------------------------------------------------------------------------------


## Review Scoring Agent
We modify the above prompt to return two scores, one for the food and one for customer service.

## Compute the final score

Routine to compute the final score and to process the summary message.

In [21]:
print(len(chat_result))
#print(chat_result[-1].summary)

2


In [1]:
!pip3 show autogen

Name: autogen
Version: 0.3.0
Summary: A programming framework for agentic AI
Home-page: https://github.com/autogen-ai/autogen
Author: Chi Wang & Qingyun Wu
Author-email: auto-gen@outlook.com
License: Apache Software License 2.0
Location: /Users/arunesh/github/llm_agents/lab01_release/.venv/lib/python3.11/site-packages
Requires: diskcache, docker, flaml, numpy, openai, packaging, pydantic, python-dotenv, termcolor, tiktoken
Required-by: 
